In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import pi
# qiskit
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)
# qurry
sys.path.insert(0, './')
from backend import backendWrapper
from mori import TagMap
print("Modules import completed...")
print("-"*30+"\n ### Init IBMQ and Set up provider")
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(
    hub='ibm-q-hub-ntu', group='ntu-internal', project='default')
backend = backendWrapper(provider)
print("IBMQ loading completed...")

Modules import completed...
------------------------------
 ### Init IBMQ and Set up provider
IBMQ loading completed...


In [15]:
Ising_matrix = [[117166.55538827574,
   39051.24837953327,
   39051.24837953327,
   39051.24837953327,
   2.5,
   0.0,
   39051.24837953327,
   3.905124837953327,
   0.0], [39051.24837953327,
   117166.55538827568,
   39051.24837953327,
   0.0,
   39051.24837953327,
   2.5,
   0.0,
   39051.24837953327,
   3.905124837953327], [39051.24837953327,
   39051.24837953327,
   117166.55538827568,
   2.5,
   0.0,
   39051.24837953327,
   3.905124837953327,
   0.0,
   39051.24837953327], [39051.24837953327,
   2.5,
   0.0,
   117164.7451385998,
   39051.24837953327,
   39051.24837953327,
   39051.24837953327,
   3.0,
   0.0], [0.0,
   39051.24837953327,
   2.5,
   39051.24837953327,
   117164.7451385998,
   39051.24837953327,
   0.0,
   39051.24837953327,
   3.0], [2.5,
   0.0,
   39051.24837953327,
   39051.24837953327,
   39051.24837953327,
   117164.7451385998,
   3.0,
   0.0,
   39051.24837953327], [39051.24837953327,
   3.905124837953327,
   0.0,
   39051.24837953327,
   3.0,
   0.0,
   117167.55538827568,
   39051.24837953327,
   39051.24837953327], [0.0,
   39051.24837953327,
   3.905124837953327,
   0.0,
   39051.24837953327,
   3.0,
   39051.24837953327,
   117167.55538827568,
   39051.24837953327], [3.905124837953327,
   0.0,
   39051.24837953327,
   3.0,
   0.0,
   39051.24837953327,
   39051.24837953327,
   39051.24837953327,
   117167.55538827568]]

### Prpepred function

In [16]:
nqubits = len(Ising_matrix)

In [17]:
from IPython.display import clear_output
from typing import Callable
from collections import defaultdict
from scipy.optimize import minimize
import torch

def H_Ising(bitstring, Ising_coeff):
    spins = []
    for bit in bitstring:
        if bit == '0':
            spins.append(-1)
        elif bit == '1':
            spins.append(1)
    H = 0
    for i in range(0, nqubits):
        H += Ising_coeff[i][i]*spins[i]
        for j in range(i+1, nqubits):
            H += Ising_coeff[i][j]*spins[i]*spins[j]
    return H


def compute_expectation(counts: dict[str, int], Ising_coeff: np.ndarray):   
    """
    Computes expectation value based on measurement results
    """   
    sum = 0
    sum_count = 0
    for bitstring, count in counts.items():
        sum += count*H_Ising(bitstring, Ising_coeff)
        sum_count += count
    return sum/sum_count


def create_qaoa_circ(
    Ising_coeff: np.ndarray, 
    theta: list[float],
) -> QuantumCircuit: 
    """
    Creates a parametrized qaoa circuit
    """   
    p = len(theta)//2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)
    
    beta = theta[:p]
    gamma = theta[p:]
    
    # initial_state
    for i in range(0, nqubits):
        qc.h(i)
    
    for irep in range(0, p):
        
        # problem unitary
        for i in range(nqubits):
            for j in range(nqubits):
                if Ising_coeff != 0 and i!=j:
                    qc.rzz(2 * Ising_coeff[i][j]*gamma[irep], i, j)
                    
        for i in range(nqubits):
            qc.rz(2 * Ising_coeff[i][i]*gamma[irep], i)
        
        # mixer unitary
        for i in range(nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc


def qaoa_circs(
    Ising_coeff: np.ndarray,
) -> Callable[[list[float]], QuantumCircuit]:
    
    def theta_depend(theta: list[float]) -> QuantumCircuit:
        return create_qaoa_circ(
            Ising_coeff=Ising_coeff, 
            theta=theta,
        )
    
    return theta_depend


def expect(
    Ising_coeff: np.ndarray,
) -> Callable[[dict[str, int]], float]:
    
    def counts_depend(counts: dict[str, int]) -> float:
        return compute_expectation(
            counts=counts, 
            Ising_coeff=Ising_coeff,
        )
    
    return counts_depend


def countCombine(countList: list[dict[str, int]]) -> dict[str, int]:
    summary = defaultdict(int)
    for c in countList:
        for k, v in c.items():
            summary[k] += v
    return summary


def expecting(Ising_matrix: np.ndarray) -> Callable[[list[float]], float]:
    qaoa_circs_giver = qaoa_circs(Ising_matrix)
    expectation_giver = expect(Ising_matrix)
    # Finally we write a function that executes the circuit on the chosen backend
    def get_expectation(theta: list[float]) -> float:
        """
        Runs parametrized circuit and get expectation value
    
        """
        # waveName = qaoaQurry.addWave(qaoa_circs_giver(theta))
        # singleIterResult = qaoaQurry.multiOutput(
        #     configList=[{ 
        #         'wave': waveName,
        #         'tags': waveName,
        #         'shots': 2**(nqubits+3),
        #         'sampling': sampling,
        #     }],
        #     backend=backend('aer_gpu'),
        #     saveLocation=Path('test.004')
        # )
        # countsList: list[dict[str, int]] = singleIterResult['tagMapCounts'][waveName]
        # counts = countCombine(countsList)
        
        qc = qaoa_circs_giver(theta)
        counts = backend('aer_gpu').run(qc, shots=2**16).result().get_counts()
        clear_output()
        print(counts)
        global lastCounts
        lastCounts = counts

        return expectation_giver(counts)
    
    return get_expectation


def maxCounts(_counts: dict[str, int]) -> str:
    vs = list(_counts.values())
    vsMax = max(_counts.values())
    vsMaxIndex = list(_counts.keys())[vs.index(vsMax)]
    return vsMaxIndex


def filterCounts(counts: dict[str, int]) -> dict[str, int]:
    bitLen = len(list(counts)[0])
    qubitLen = int(np.sqrt(bitLen))
    if (qubitLen % 1 > 0):
        raise ValueError(f"bitLen: {bitLen}, qubitLen: {qubitLen}.")

    def filter(bitstr: str) -> bool:
        strList = [bitstr[qubitLen*(k):qubitLen*(k+1)] for k in range(qubitLen)]
        strList1Sum = [sum([int(b) for b in bs]) for bs in strList]
        strList1PosImmutable = dict.fromkeys(strList)
            
        return all([ s==1 for s in strList1Sum]) and len(strList1PosImmutable) == qubitLen
    
    filteredCounts = {
        k: v
    for k, v in counts.items() if filter(k) }
    
    return filteredCounts


def thetaGen(length: int) -> list[float]:
    torch.manual_seed(0)
    theta = torch.empty(length*2)
    torch.nn.init.normal_(theta, mean=np.pi, std=0.01*np.pi)
    theta = theta.tolist()
    
    return theta

### Exp. Unit

In [18]:
allLastCounts = TagMap()
allLastCountsAvailable = TagMap()
allRes = TagMap()
allLastCountsInfo = TagMap()

lastCounts = {}

In [19]:
def expUnit(catol: float, depth: int):
    expectation = expecting(Ising_matrix=Ising_matrix)
    res = minimize(
        expectation, 
        thetaGen(depth), 
        options = {
            'iprint': 1, 
            'disp': False, 
            'maxiter': 10000,
            'catol': catol, 
            'rhobeg': 5.0
        },
        method='COBYLA'
    )
    allLastCounts[(f'catol={catol}', f'depth={depth}')].append(lastCounts)
    lastFilter = filterCounts(lastCounts)
    allLastCountsAvailable[(f'catol={catol}', f'depth={depth}')].append(lastFilter)
    
    allRes[(f'catol={catol}', f'depth={depth}')].append(res)
    allLastCountsInfo[(f'catol={catol}', f'depth={depth}')].append({
        'allShots': sum(lastCounts.values()),
        'available': sum(lastFilter.values()),
        'ratio': sum(lastFilter.values())/sum(lastCounts.values()),
    })
    

In [20]:
catolList = [0.0002, 0.002, 0.02, 0.2, 2]
depthList = range(9)

In [21]:
configList = [
    {
        'catol': catol, 
        'depth': depth
    }
    for catol in catolList
    for depth in depthList
]

for config in configList:
    expUnit(**config)

{'100010101': 6, '001100011': 11, '101100101': 6, '001011101': 38, '101110000': 27, '011100100': 21, '110100110': 10, '110101001': 21, '010100100': 13, '100001110': 13, '110111111': 27, '111001110': 36, '111001111': 34, '100011010': 54, '110100111': 27, '111010011': 22, '110011001': 23, '111010001': 39, '000110101': 35, '010101110': 42, '110101000': 38, '100111000': 92, '111100110': 22, '111001000': 29, '100100110': 93, '100011111': 45, '110011010': 30, '011101011': 47, '101010101': 33, '010110011': 48, '000110000': 101, '001000111': 167, '010101010': 34, '111011110': 50, '101000101': 29, '100110110': 118, '010001110': 6, '110101100': 39, '111100101': 29, '111001101': 17, '011001011': 6, '100010010': 18, '010110111': 51, '100110010': 50, '111011011': 45, '111010101': 34, '001011000': 75, '111001100': 30, '011101100': 32, '101101111': 64, '001110010': 56, '011010011': 9, '001000101': 68, '111001010': 35, '000011111': 55, '010010100': 12, '100110001': 61, '101100111': 35, '010011101': 35

In [22]:
# allLastCountsAvailable = TagMap()
# allLastCountsInfo = TagMap()

# def refiliter(catol: float, depth: int):

#     lastFilter = filterCounts(allLastCounts[(f'catol={catol}', f'depth={depth}')][0])
#     allLastCountsAvailable[(f'catol={catol}', f'depth={depth}')].append(lastFilter)
#     allLastCountsInfo[(f'catol={catol}', f'depth={depth}')].append({
#         'allShots': sum(lastCounts.values()),
#         'available': sum(lastFilter.values()),
#         'ratio': sum(lastFilter.values())/sum(lastCounts.values()),
#     })


# for config in configList:
#     refiliter(**config)
#     # print(config)

In [23]:
expsName = Path('test.p=10000.3nodes.001')
if not os.path.exists(expsName):
    os.mkdir(expsName)

allLastCounts.export(
    saveLocation=expsName,
    additionName=expsName,
    name='allLastCounts'
)
allLastCountsAvailable.export(
    saveLocation=expsName,
    additionName=expsName,
    name='allLastCountsAvailable'
)
allRes.export(
    saveLocation=expsName,
    additionName=expsName,
    name='allRes'
)
allLastCountsInfo.export(
    saveLocation=expsName,
    additionName=expsName,
    name='allLastCountsInfo'
)


PosixPath('test.p=10000.3nodes.001/test.p=10000.3nodes.001.allLastCountsInfo.json')

In [24]:
filterCounts({'010010010': 33, '100010001': 33})

{'100010001': 33}

### Read